In [1]:
from numpy import pi, cos, sin, sqrt
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, Point
import pandas as pd
from math import modf
import json

hex_width = 200
city_radius = 0.20 # in lat-long degrees

hex_size = city_radius/(2*hex_width)

In [2]:
df = gpd.GeoDataFrame(pd.read_pickle('../transit/points_pickle'),crs={'init' :'epsg:3857'})
df.head(1)['scores'].values

array([{'score': 77, 'bikeshare': '0%', 'carshare': '0%', 'masstransit': '0%', 'ridehailing': '74%'}],
      dtype=object)

In [4]:
df = gpd.GeoDataFrame(pd.read_pickle('../transit/points_pickle'),crs={'init' :'epsg:3857'})
df['total_score'] = df['scores'].apply(lambda x: float(x['score']))
df['bike_score'] = df['scores'].apply(lambda x: float(x['bikeshare'].strip('%')))
df['carshare_score'] = df['scores'].apply(lambda x: float(x['carshare'].strip('%')))
df['mass_transit_score'] = df['scores'].apply(lambda x: float(x['masstransit'].strip('%')))
df['ride_hail_score'] = df['scores'].apply(lambda x: float(x['ridehailing'].strip('%')))
df.head()

,X,Y,UNITNUM,scores,geometry,shape_id,total_score,bike_score,carshare_score,mass_transit_score,ride_hail_score
ADDRESS_ID,,,,,,,,,,,
243225,-77.037265,38.936436,1181.0,"{'score': 77, 'bikeshare': '0%', 'carshare': '...",POINT (-77.03726510819578 38.93643575527931),8,77.0,0.0,0.0,0.0,74.0
284325,-77.024650,38.920318,917.0,"{'score': 73, 'bikeshare': '0%', 'carshare': '...",POINT (-77.02464966400156 38.92031826898436),8,73.0,0.0,0.0,0.0,70.0
294461,-77.081959,38.935867,671.0,"{'score': 79, 'bikeshare': '19%', 'carshare': ...",POINT (-77.08195893397365 38.93586687687617),8,79.0,19.0,19.0,5.0,52.0
235844,-77.035880,38.923626,664.0,"{'score': 79, 'bikeshare': '19%', 'carshare': ...",POINT (-77.03588045315645 38.92362582942218),8,79.0,19.0,19.0,6.0,51.0
220630,-77.061250,38.943732,638.0,"{'score': 79, 'bikeshare': '19%', 'carshare': ...",POINT (-77.06125006503876 38.94373173501572),8,79.0,19.0,19.0,6.0,51.0


In [5]:
def axial_to_cube(x,y):
    z = -x -y
    return (x,y,z)

In [6]:


"""function hex_to_pixel(hex):
    x = size * 3/2 * hex.q
    y = size * sqrt(3) * (hex.r + hex.q/2)
    return Point(x, y)"""



class hexagon:
    def __init__(self, coords, city_center=(38.904978, -77.039658), size=hex_size):
        # there will be no coordinate reference system at the 
        # hex level, that needs to be handled by the grid
        self.x_loc = float(coords[0])
        self.y_loc = float(coords[1])
        self.z_loc = float(coords[2])
        self.city_center = city_center
        self.size = size
        self.self_center()
        self.create_vertices()
    def self_center(self):
        self.x_center = self.size * 1.5 * self.x_loc
        self.y_center = self.size * sqrt(3) * (self.y_loc + self.x_loc/2)        
    def create_vertices(self):
        # I am only interested in making "flat top" layouts
        self.vertices = tuple(self.vertex(n) for n in range(6))
    def vertex(self, n):
        angle = n*pi/3
        i = self.size*cos(angle) + self.x_center + self.city_center[1]
        j = self.size*sin(angle) + self.y_center + self.city_center[0]
        return(i,j)

In [7]:


def xy_generator(center,grid,size):
    cx = grid[0] * 1.5 * size + center[1] # using lat-long convention for the city centers so x is the second value in center but not grid
    cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + center[0]
    x=[]
    y=[]
    
    for n in range(6):
        angle=n*pi/3
        x.append(size*cos(angle)+cx)
        y.append(size*sin(angle)+cy)
    return (x,y)

xy_generator((39,-77),(0,0),0.025)



([-76.975, -76.9875, -77.0125, -77.025, -77.0125, -76.9875],
 [39.0,
  39.02165063509461,
  39.02165063509461,
  39.0,
  38.97834936490539,
  38.97834936490539])

In [8]:


class hexagonal_grid_axial:
    def __init__(self, width, size=0.0025, coordinates = (38.904978, -77.039658)):
        self.width=width
        self.size=size
        self.coordinates = coordinates
        self.hex_dataframe_creator()
    def set_crs(self, crs):
        pass
    def make_ring(self,n):
        ring = []
        for m in range(1,n+1):
            ring.append(((n,-m,-n+m),m))
            ring.append(((-m,-n+m,n),m))
            ring.append(((-n+m,n,-m),m))
            ring.append(((-n,m, n-m),m))
            ring.append(((m,n-m,-n),m))
            ring.append(((n-m,-n,m),m))
        return ring
    
    def fill_grid(self):
        hex_list = [((0,0,0),0)]
        for n in range(1,self.width+1):
            hex_list+=self.make_ring(n)
        return hex_list
        
    def y_generator(self, grid):
        center = self.coordinates
        size = self.size
        cy = size * sqrt(3) * ( grid[1] + grid[0]/2 ) + center[0]
        # using lat-long convention for the city centers so x is the second value in center but not grid
        y=[]
        for n in range(6):
            angle=n*pi/3
            y.append(size*sin(angle)+cy)
        return y
    
    def x_generator(self,grid):
        center = self.coordinates
        size = self.size
        cx = 1.5 * float(grid[0]) * size + center[1] 
        # using lat-long convention for the city centers so x is the second value in center but not grid
        x=[]

        for n in range(6):
            angle=n*pi/3
            x.append(size*cos(angle)+cx)
        return x
    
    def hex_dataframe_creator(self):
        self.hexes = gpd.GeoDataFrame(self.fill_grid(), columns = ['hexagon','radius'])
        self.hexes['y'] = self.hexes.hexagon.apply(self.y_generator)
        self.hexes['x'] = self.hexes.hexagon.apply(self.x_generator)




        
df_hex = hexagonal_grid_axial(hex_width,hex_size).hexes
df_hex.head()

,hexagon,radius,y,x
0,"(0, 0, 0)",0,"[38.904978, 38.90541101270189, 38.905411012701...","[-77.039158, -77.03940800000001, -77.039908, -..."
1,"(1, -1, 0)",1,"[38.90454498729811, 38.904978, 38.904978, 38.9...","[-77.038408, -77.03865800000001, -77.039158, -..."
2,"(-1, 0, 1)",1,"[38.90454498729811, 38.904978, 38.904978, 38.9...","[-77.039908, -77.040158, -77.040658, -77.04090..."
3,"(0, 1, -1)",1,"[38.90584402540379, 38.90627703810568, 38.9062...","[-77.039158, -77.03940800000001, -77.039908, -..."
4,"(-1, 1, 0)",1,"[38.90541101270189, 38.90584402540378, 38.9058...","[-77.039908, -77.040158, -77.040658, -77.04090..."


In [9]:
def hex_round(x,y):
    z = -x -y
    x_d, x = modf(x)
    y_d, y = modf(y)
    z_d, z = modf(z)
    
    if x_d>y_d and x_d>z_d:
        x = -y-z
    elif y_d>z_d:
        y = -x-z
    else:
        z=-x-y
        
    return (int(x),int(y),int(z))

In [10]:
ssq = sqrt(3)/3

def point_loc_to_hex_loc(x_h, y_h, size):
    x = (x_h * 2/3)/ size
    y = ((-x_h/3) + (ssq * y_h)) / size
    return hex_round(x,y)


In [11]:
def hex_center(x_row, y_row, size):
    x = size * 3/2 * x_row
    y = size * (3**0.5) * (y_row + x_row/2)
    return x, y

In [12]:
washington_y = 38.904978
washington_x = -77.039658
hex_assign = lambda p: point_loc_to_hex_loc(p.x-washington_x, p.y-washington_y, hex_size)


In [13]:
df['hexagon'] = df['geometry'].apply(hex_assign)


df.head()



,X,Y,UNITNUM,scores,geometry,shape_id,total_score,bike_score,carshare_score,mass_transit_score,ride_hail_score,hexagon
ADDRESS_ID,,,,,,,,,,,,
243225,-77.037265,38.936436,1181.0,"{'score': 77, 'bikeshare': '0%', 'carshare': '...",POINT (-77.03726510819578 38.93643575527931),8,77.0,0.0,0.0,0.0,74.0,"(3, 34, -37)"
284325,-77.024650,38.920318,917.0,"{'score': 73, 'bikeshare': '0%', 'carshare': '...",POINT (-77.02464966400156 38.92031826898436),8,73.0,0.0,0.0,0.0,70.0,"(20, 7, -27)"
294461,-77.081959,38.935867,671.0,"{'score': 79, 'bikeshare': '19%', 'carshare': ...",POINT (-77.08195893397365 38.93586687687617),8,79.0,19.0,19.0,5.0,52.0,"(-56, 63, -7)"
235844,-77.035880,38.923626,664.0,"{'score': 79, 'bikeshare': '19%', 'carshare': ...",POINT (-77.03588045315645 38.92362582942218),8,79.0,19.0,19.0,6.0,51.0,"(5, 19, -24)"
220630,-77.061250,38.943732,638.0,"{'score': 79, 'bikeshare': '19%', 'carshare': ...",POINT (-77.06125006503876 38.94373173501572),8,79.0,19.0,19.0,6.0,51.0,"(-28, 58, -30)"


In [16]:

df_grouped = gpd.GeoDataFrame(df.groupby('hexagon', sort=True)[['total_score',
            'bike_score','carshare_score','mass_transit_score',
            'ride_hail_score']].mean(),crs='epsg:3857').reset_index()

In [37]:
df['count']=1

df1 = df.groupby('hexagon', sort=True,as_index=False)[['total_score',
            'bike_score','carshare_score','mass_transit_score',
            'ride_hail_score']].mean()

df2 = df.groupby('hexagon', sort=True, as_index=False)[['count']].sum()

df_grouped = df1.merge(df2)

# print(df_grouped.head(), df2.head())

In [38]:
df_grouped.head()

,hexagon,total_score,bike_score,carshare_score,mass_transit_score,ride_hail_score,count
0,"(-95, 79, 16)",63.0,0.0,0.0,1.0,51.0,1
1,"(-95, 84, 11)",78.0,23.0,23.0,1.0,51.0,1
2,"(-93, 79, 14)",76.0,0.0,0.0,6.0,66.0,1
3,"(-84, 66, 18)",63.5,0.0,0.0,6.0,47.0,2
4,"(-84, 67, 17)",80.0,0.0,0.0,0.0,77.0,1


In [39]:
df_merged = df_hex.merge(df_grouped)

In [40]:
df_merged.head()

,hexagon,radius,y,x,total_score,bike_score,carshare_score,mass_transit_score,ride_hail_score,count
0,"(1, 2, -3)",1,"[38.90714306350946, 38.90757607621135, 38.9075...","[-77.038408, -77.03865800000001, -77.039158, -...",81.0,0.0,0.0,6.0,71.0,1
1,"(3, -3, 0)",3,"[38.90367896189432, 38.90411197459621, 38.9041...","[-77.036908, -77.037158, -77.037658, -77.03790...",79.0,20.0,20.0,5.0,51.0,1
2,"(-3, 3, 0)",3,"[38.90627703810568, 38.90671005080757, 38.9067...","[-77.041408, -77.04165800000001, -77.042158, -...",80.0,14.0,14.0,10.0,52.0,1
3,"(3, 0, -3)",3,"[38.90627703810568, 38.90671005080757, 38.9067...","[-77.036908, -77.037158, -77.037658, -77.03790...",86.0,18.0,18.0,7.0,56.0,1
4,"(1, 3, -4)",1,"[38.908009088913246, 38.90844210161514, 38.908...","[-77.038408, -77.03865800000001, -77.039158, -...",85.0,15.0,15.0,7.0,58.0,1


In [41]:
# df_merged[['geometry','base_score']].to_pickle('wash_once_more_pickle')

df_merged.drop(['radius'],axis=1).to_pickle('wash_once_more_pickle')

In [42]:
# hex_json = df_hex[['geometry']].to_json()
# with open('wash_again.json', 'w') as outfile:
#     json.dump(hex_json, outfile)

In [714]:
df_merged['base_score'].describe()

count    1157.000000
mean       74.830808
std         5.339849
min        55.000000
25%        72.111111
50%        75.000000
75%        77.750000
max        95.000000
Name: base_score, dtype: float64

In [43]:
df_merged.shape

(5849, 10)